# Section 2: Testing

### 9.1 Load the Model

In [ ]:
model_path = 'new_results2/current_model_800.pth'
current_model = RLModel(agent_configs['network_arch'])
checkpoint = torch.load(model_path)
current_model.load_state_dict(checkpoint['model_state_dict'])

### 9.2 Softmax policy

In [ ]:
def policy(state, model, num_actions = 4):
        
        """
        Select the action given a single state.
        
        """
        model.eval()
        # compute action values states:(1, state_dim)
        q_values = model(state)

        # compute the probs of each action (1, num_actions)
        probs = softmax(q_values.data, tau = 0.01)
        probs = np.array(probs)
        probs /= probs.sum()

        # select action
        rand_generator = np.random.RandomState(seed = 1)
        action = rand_generator.choice(num_actions, 1, p = probs.squeeze())
#         action = np.argmax(probs.squeeze())
        
        return action

### 9.3 Visualize the results

In [ ]:
env = gym.make("LunarLander-v2")
env = wrappers.Monitor(env, './videos_800/' + '/')
for i_episode in range(10):
    observation = env.reset()
    total_reward = 0
    
    for t in range(1000):
        env.render()
#         print(observation)
        with torch.no_grad():
            observation = Variable(torch.tensor(observation).view(1, -1))
            action = policy(observation, current_model)
            action = int(action.squeeze())

            observation, reward, done, info = env.step(action)
            total_reward += reward
            if done:
                
                print("Episode finished after {} timesteps, total reward : {}".format(t+1, total_reward))
                break
env.close()
print('end')